<a href="https://colab.research.google.com/github/MuhammadA15/BattleShip-Game/blob/master/ML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install basketball-reference-scraper==v1.0.1

In [ ]:
!pip install sportsreference

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sportsreference.nba.teams import Teams
from sportsreference.nba.boxscore import Boxscore
from tqdm import tqdm
import gc
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from basketball_reference_scraper.teams import get_roster, get_team_stats, get_opp_stats, get_roster_stats, get_team_misc
from basketball_reference_scraper.box_scores import get_box_scores
from basketball_reference_scraper.seasons import get_schedule, get_standings
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [ ]:
season1 = range(2016,2017)
season2 = range(2017,2018)
season3 = range(2018,2019)
season4 = range(2019,2020)
box_df1 = None
box_df2 = None
box_df3 = None
box_df4 = None
sched_df1 = None
sched_df2 = None
sched_df3 = None
sched_df4 = None
count = 1
teams = ['ATL','BRK','BOS','CHI','CLE','DAL','DEN','DET','GSW','HOU','IND','LAC',
         'LAL','MEM','MIA','MIL','MIN','NOP','NYK','OKC','ORL','PHI','PHO','POR','SAC','SAS','TOR','UTA','WAS']

options = {'Atlanta Hawks': 'ATL','Brooklyn Nets': 'BRK', 'Boston Celtics': 'BOS', 'Chicago Bulls': 'CHI', 'Cleveland Cavaliers': 'CLE',
            'Dallas Mavericks': 'DAL', 'Denver Nuggets': 'DEN', 'Detroit Pistons': 'DET', 'Golden State Warriors': 'GSW', 'Houston Rockets': 'HOU', 'Indiana Pacers': 'IND', 
            'Los Angeles Clippers': 'LAC', 'Los Angeles Lakers': 'LAL', 'Memphis Grizzlies': 'MEM', 'Miami Heat': 'MIA', 'Milwaukee Bucks': 'MIL', 'Minnesota Timberwolves': 'MIN', 
           'New Orleans Pelicans': 'NOP', 'New York Knicks': 'NYK', 'Oklahoma City Thunder': 'OKC', 'Orlando Magic': 'ORL', 'Philadelphia 76ers': 'PHI', 'Phoenix Suns': 'PHO', 
           'Portland Trail Blazers': 'POR', 'Sacramento Kings': 'SAC', 'San Antonio Spurs': 'SAS', 'Toronto Raptors': 'TOR', 'Utah Jazz': 'UTA', 'Washington Wizards': 'WAS'}

for season in season1:
    d = get_schedule(season, playoffs=False)
    d = d[(d.HOME != 'Charlotte Hornets') & (d.VISITOR != 'Charlotte Hornets') & (d.HOME != 'Charlotte Bobcats') & (d.VISITOR != 'Charlotte Bobcats')]
    for i, row in d.iterrows():
      #print(row.HOME + " vs " + row.VISITOR)
      t = options.get(row.HOME, 'CHO')
      df1 = get_team_stats(t, season, data_format='PER_GAME')
      df1 = df1.to_frame()
      df1 = df1.transpose()
      df1.columns = [str(col) + '_x' for col in df1.columns]

      df2 = get_opp_stats(t, season, data_format='PER_GAME')
      df2 = df2.to_frame()
      df2 = df2.transpose()
      df2.columns = [str(col) + '_x' for col in df2.columns]

      o = options.get(row.VISITOR, 'CHO')
      df3 = get_team_stats(o, season, data_format='PER_GAME')
      df3 = df3.to_frame()
      df3 = df3.transpose()
      df3.columns = [str(col) + '_y' for col in df3.columns]

      df4 = get_opp_stats(o, season, data_format='PER_GAME')
      df4 = df4.to_frame()
      df4 = df4.transpose()
      df4.columns = [str(col) + '_y' for col in df4.columns]

      f = row.to_frame()
      f = f.transpose()

      df1.reset_index(drop=True, inplace=True)
      df2.reset_index(drop=True, inplace=True)
      df3.reset_index(drop=True, inplace=True)
      df4.reset_index(drop=True, inplace=True)
      f.reset_index(drop=True, inplace=True)
      full_row = pd.concat([f, df1, df2, df3, df4],axis=1)

      if box_df1 is not None:
        box_df1 = pd.concat([box_df1, full_row],ignore_index=True,axis=0)
      else:
        box_df1 = full_row   
      print(count)
      count += 1

In [ ]:
for season in season2:
    d2 = get_schedule(season, playoffs=False)
    d2 = d2[(d2.HOME != 'Charlotte Hornets') & (d2.VISITOR != 'Charlotte Hornets') & (d2.HOME != 'Charlotte Bobcats') & (d2.VISITOR != 'Charlotte Bobcats')]
    for i, row in d2.iterrows():
      #print(row.HOME + " vs " + row.VISITOR)
      t2 = options.get(row.HOME, 'CHO')
      df12 = get_team_stats(t2, season, data_format='PER_GAME')
      df12 = df12.to_frame()
      df12 = df12.transpose()
      df12.columns = [str(col) + '_x' for col in df12.columns]

      df22 = get_opp_stats(t2, season, data_format='PER_GAME')
      df22 = df22.to_frame()
      df22 = df22.transpose()
      df22.columns = [str(col) + '_x' for col in df22.columns]

      o2 = options.get(row.VISITOR, 'CHO')
      df32 = get_team_stats(o2, season, data_format='PER_GAME')
      df32 = df32.to_frame()
      df32 = df32.transpose()
      df32.columns = [str(col) + '_y' for col in df32.columns]

      df42 = get_opp_stats(o2, season, data_format='PER_GAME')
      df42 = df42.to_frame()
      df42 = df42.transpose()
      df42.columns = [str(col) + '_y' for col in df42.columns]

      f2 = row.to_frame()
      f2 = f2.transpose()

      df12.reset_index(drop=True, inplace=True)
      df22.reset_index(drop=True, inplace=True)
      df32.reset_index(drop=True, inplace=True)
      df42.reset_index(drop=True, inplace=True)
      f2.reset_index(drop=True, inplace=True)
      full_row2 = pd.concat([f2, df12, df22, df32, df42],axis=1)

      if box_df2 is not None:
        box_df2 = pd.concat([box_df2, full_row2],ignore_index=True,axis=0)
      else:
        box_df2 = full_row2   
      print(count)
      count += 1

In [ ]:
for season in season3:
    d3 = get_schedule(season, playoffs=False)
    d3 = d3[(d3.HOME != 'Charlotte Hornets') & (d3.VISITOR != 'Charlotte Hornets') & (d3.HOME != 'Charlotte Bobcats') & (d3.VISITOR != 'Charlotte Bobcats')]
    for i, row in d3.iterrows():
      #print(row.HOME + " vs " + row.VISITOR)
      t3 = options.get(row.HOME, 'CHO')
      df13 = get_team_stats(t3, season, data_format='PER_GAME')
      df13 = df13.to_frame()
      df13 = df13.transpose()
      df13.columns = [str(col) + '_x' for col in df13.columns]

      df23 = get_opp_stats(t3, season, data_format='PER_GAME')
      df23 = df23.to_frame()
      df23 = df23.transpose()
      df23.columns = [str(col) + '_x' for col in df23.columns]

      o3 = options.get(row.VISITOR, 'CHO')
      df33 = get_team_stats(o3, season, data_format='PER_GAME')
      df33 = df33.to_frame()
      df33 = df33.transpose()
      df33.columns = [str(col) + '_y' for col in df33.columns]

      df43 = get_opp_stats(o3, season, data_format='PER_GAME')
      df43 = df43.to_frame()
      df43 = df43.transpose()
      df43.columns = [str(col) + '_y' for col in df43.columns]

      f3 = row.to_frame()
      f3 = f3.transpose()

      df13.reset_index(drop=True, inplace=True)
      df23.reset_index(drop=True, inplace=True)
      df33.reset_index(drop=True, inplace=True)
      df43.reset_index(drop=True, inplace=True)
      f3.reset_index(drop=True, inplace=True)
      full_row3 = pd.concat([f3, df13, df23, df33, df43],axis=1)

      if box_df3 is not None:
        box_df3 = pd.concat([box_df3, full_row3],ignore_index=True,axis=0)
      else:
        box_df3 = full_row3   
      print(count)
      count += 1

In [ ]:
box_df5 = None
for season in season4:
    d5 = get_schedule(season, playoffs=False)
    d5 = d5[(d5.HOME != 'Charlotte Hornets') & (d5.VISITOR != 'Charlotte Hornets') & (d5.HOME != 'Charlotte Bobcats') & (d5.VISITOR != 'Charlotte Bobcats')]
    for i, row in d5.iterrows():
      #print(row.HOME + " vs " + row.VISITOR)
      t5 = options.get(row.HOME, 'CHO')
      df15 = get_team_stats(t5, season, data_format='PER_GAME')
      df15 = df15.to_frame()
      df15 = df15.transpose()
      df15.columns = [str(col) + '_x' for col in df15.columns]

      df25 = get_opp_stats(t5, season, data_format='PER_GAME')
      df25 = df25.to_frame()
      df25 = df25.transpose()
      df25.columns = [str(col) + '_x' for col in df25.columns]

      o5 = options.get(row.VISITOR, 'CHO')
      df35 = get_team_stats(o5, season, data_format='PER_GAME')
      df35 = df35.to_frame()
      df35 = df35.transpose()
      df35.columns = [str(col) + '_y' for col in df35.columns]

      df45 = get_opp_stats(o5, season, data_format='PER_GAME')
      df45 = df45.to_frame()
      df45 = df45.transpose()
      df45.columns = [str(col) + '_y' for col in df45.columns]

      f5 = row.to_frame()
      f5 = f5.transpose()

      df15.reset_index(drop=True, inplace=True)
      df25.reset_index(drop=True, inplace=True)
      df35.reset_index(drop=True, inplace=True)
      df45.reset_index(drop=True, inplace=True)
      f5.reset_index(drop=True, inplace=True)
      full_row5 = pd.concat([f5, df15, df25, df35, df45],axis=1)

      if box_df5 is not None:
        box_df5 = pd.concat([box_df5, full_row5],ignore_index=True,axis=0)
      else:
        box_df5 = full_row5   
      print(count)
      count += 1

In [ ]:
box_df = pd.concat([box_df1, box_df2, box_df3],ignore_index=True,axis=0)

In [ ]:
box_df

,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS,G_x,MP_x,FG_x,FGA_x,FG%_x,3P_x,3PA_x,3P%_x,2P_x,2PA_x,2P%_x,FT_x,FTA_x,FT%_x,ORB_x,DRB_x,TRB_x,AST_x,STL_x,BLK_x,TOV_x,PF_x,PTS_x,TEAM_x,SEASON_x,OPP_G_x,OPP_MP_x,OPP_FG_x,OPP_FGA_x,OPP_FG%_x,OPP_3P_x,OPP_3PA_x,OPP_3P%_x,OPP_2P_x,OPP_2PA_x,...,2P%_y,FT_y,FTA_y,FT%_y,ORB_y,DRB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y,PTS_y,TEAM_y,SEASON_y,OPP_G_y,OPP_MP_y,OPP_FG_y,OPP_FGA_y,OPP_FG%_y,OPP_3P_y,OPP_3PA_y,OPP_3P%_y,OPP_2P_y,OPP_2PA_y,OPP_2P%_y,OPP_FT_y,OPP_FTA_y,OPP_FT%_y,OPP_ORB_y,OPP_DRB_y,OPP_TRB_y,OPP_AST_y,OPP_STL_y,OPP_BLK_y,OPP_TOV_y,OPP_PF_y,OPP_PTS_y,TEAM_y,SEASON_y
0,2015-10-27,Cleveland Cavaliers,95,Chicago Bulls,97,82,242.7,38.6,87.4,0.441,7.9,21.4,0.371,30.7,66.1,0.464,16.5,21,0.787,11.1,35.2,46.3,22.8,6,5.7,13.9,18.8,101.6,CHI,2015-16,82,242.7,39.4,89.4,0.441,7.9,23,0.345,31.5,66.4,...,0.514,16.3,21.7,0.748,10.6,33.9,44.5,22.7,6.7,3.9,13.6,20.3,104.3,CLE,2015-16,82,242.1,36.8,82.1,0.448,7.9,22.7,0.347,28.9,59.4,0.487,16.8,22.6,0.743,9.3,31.8,41,21.4,7.2,4.4,13.3,20.6,98.3,CLE,2015-16
1,2015-10-27,Detroit Pistons,106,Atlanta Hawks,94,82,241.8,38.6,84.4,0.458,9.9,28.4,0.35,28.7,56.1,0.512,15.6,20,0.783,8.3,33.8,42.1,25.6,9.1,5.9,15,19.1,102.8,ATL,2015-16,82,241.8,37.1,86.1,0.432,8.3,24.5,0.338,28.9,61.6,...,0.48,17.1,25.5,0.668,12.5,33.9,46.3,19.4,7,3.7,13.5,19,102,DET,2015-16,82,242.4,38.8,84.2,0.461,7.3,20.5,0.355,31.5,63.7,0.495,16.5,21,0.783,8.8,33.7,42.5,21.5,7.1,4.5,13.4,21.6,101.4,DET,2015-16
2,2015-10-27,New Orleans Pelicans,95,Golden State Warriors,111,82,242.4,42.5,87.3,0.487,13.1,31.6,0.416,29.4,55.7,0.528,16.7,21.8,0.763,10,36.2,46.2,28.9,8.4,6.1,15.2,20.7,114.9,GSW,2015-16,82,242.4,38.9,89.4,0.435,7.8,23.5,0.332,31.1,65.9,...,0.482,17.3,22.2,0.776,9.5,33.1,42.6,22.2,7.7,4.2,13.4,20.9,102.7,NOP,2015-16,82,241.2,39.2,83.9,0.468,9.2,24.9,0.369,30,59,0.51,18.8,24,0.783,8.9,35.5,44.4,23.2,7.3,5.2,13.7,19.9,106.5,NOP,2015-16
3,2015-10-28,Washington Wizards,88,Orlando Magic,87,82,242.7,39.5,86.8,0.455,7.8,22.2,0.35,31.8,64.7,0.492,15.2,20.1,0.757,10.3,33,43.3,23.6,8.2,5.1,14.1,20.7,102.1,ORL,2015-16,82,242.7,38.4,83.5,0.46,8.9,24.9,0.359,29.5,58.6,...,0.501,16.5,22.5,0.73,9.1,32.8,41.8,24.5,8.6,3.9,14.5,20.8,104.1,WAS,2015-16,82,240.9,38.7,83.8,0.462,9,24.2,0.371,29.7,59.5,0.498,18.3,23.7,0.769,9.4,34.9,44.3,22.5,8.1,4.3,16.2,20.1,104.6,WAS,2015-16
4,2015-10-28,Philadelphia 76ers,95,Boston Celtics,112,82,241.2,39.2,89.2,0.439,8.7,26.1,0.335,30.5,63.1,0.483,18.5,23.5,0.788,11.6,33.3,44.9,24.2,9.2,4.2,13.7,21.9,105.7,BOS,2015-16,82,241.2,37.5,85,0.441,7.8,23.3,0.336,29.7,61.7,...,0.476,15.7,22.6,0.694,9.5,31.8,41.2,21.5,8.3,6,16.4,21.7,97.4,PHI,2015-16,82,241.5,39.7,85.4,0.464,7.8,21.7,0.359,31.9,63.8,0.5,20.5,26.1,0.786,11.1,36.4,47.6,23.4,8.7,5.7,15.1,19.2,107.6,PHI,2015-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3439,2018-04-11,Washington Wizards,92,Orlando Magic,101,82,240.6,38.8,85.9,0.452,10.3,29.3,0.351,28.5,56.5,0.504,15.5,20.5,0.757,8.8,32.8,41.6,23.4,7.6,4.9,14.5,19.3,103.4,ORL,2017-18,82,240.6,40.9,87.5,0.468,10,27.6,0.362,30.9,60,...,0.508,16.8,21.8,0.772,10,33.1,43.1,25.2,7.9,4.3,14.6,21.3,106.6,WAS,2017-18,82,242.1,39,84.4,0.462,10.2,29.2,0.349,28.8,55.2,0.522,17.9,23.5,0.761,9.8,32.7,42.5,22.3,7.8,4.7,14.9,20,106,WAS,2017-18
3440,2018-04-11,Milwaukee Bucks,95,Philadelphia 76ers,130,82,241.2,40.8,86.6,0.472,11,29.8,0.369,29.9,56.7,0.526,17.1,22.8,0.752,10.9,36.5,47.4,27.1,8.3,5.1,16.5,22.1,109.8,PHI,2017-18,82,241.2,38,87.6,0.434,10.1,29.6,0.342,27.9,58,...,0.531,18.3,23.4,0.783,8.4,31.5,39.8,23.2,8.8,5.4,13.8,21.4,106.5,MIL,2017-18,82,241.8,39.1,83.7,0.468,10.1,27.1,0.372,29,56.6,0.513,18.4,23.5,0.783,10,32.8,42.7,23.6,7.1,4.5,15.6,19.9,106.8,MIL,2017-18
3441,2018-04-11,Los Angeles 

In [ ]:
for i, row in box_df.iterrows():
  if row.VISITOR_PTS > row.HOME_PTS:
    box_df.loc[i,'winning_name'] = row.VISITOR
  else:
    box_df.loc[i,'winning_name'] = row.HOME

In [ ]:
for i, row in box_df.iterrows():
  if row.HOME == row.winning_name:
    box_df.loc[i,'Class'] = 1
  else:
    box_df.loc[i,'Class'] = 0


In [ ]:
for i, row in box_df5.iterrows():
  if row.VISITOR_PTS > row.HOME_PTS:
    box_df5.loc[i,'winning_name'] = row.VISITOR
  else:
    box_df5.loc[i,'winning_name'] = row.HOME

for i, row in box_df5.iterrows():
  if row.HOME == row.winning_name:
    box_df5.loc[i,'Class'] = 1
  else:
    box_df5.loc[i,'Class'] = 0

In [ ]:
box_df

,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS,G_x,MP_x,FG_x,FGA_x,FG%_x,3P_x,3PA_x,3P%_x,2P_x,2PA_x,2P%_x,FT_x,FTA_x,FT%_x,ORB_x,DRB_x,TRB_x,AST_x,STL_x,BLK_x,TOV_x,PF_x,PTS_x,TEAM_x,SEASON_x,OPP_G_x,OPP_MP_x,OPP_FG_x,OPP_FGA_x,OPP_FG%_x,OPP_3P_x,OPP_3PA_x,OPP_3P%_x,OPP_2P_x,OPP_2PA_x,...,FTA_y,FT%_y,ORB_y,DRB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y,PTS_y,TEAM_y,SEASON_y,OPP_G_y,OPP_MP_y,OPP_FG_y,OPP_FGA_y,OPP_FG%_y,OPP_3P_y,OPP_3PA_y,OPP_3P%_y,OPP_2P_y,OPP_2PA_y,OPP_2P%_y,OPP_FT_y,OPP_FTA_y,OPP_FT%_y,OPP_ORB_y,OPP_DRB_y,OPP_TRB_y,OPP_AST_y,OPP_STL_y,OPP_BLK_y,OPP_TOV_y,OPP_PF_y,OPP_PTS_y,TEAM_y,SEASON_y,winning_name,Class
0,2015-10-27,Cleveland Cavaliers,95,Chicago Bulls,97,82,242.7,38.6,87.4,0.441,7.9,21.4,0.371,30.7,66.1,0.464,16.5,21,0.787,11.1,35.2,46.3,22.8,6,5.7,13.9,18.8,101.6,CHI,2015-16,82,242.7,39.4,89.4,0.441,7.9,23,0.345,31.5,66.4,...,21.7,0.748,10.6,33.9,44.5,22.7,6.7,3.9,13.6,20.3,104.3,CLE,2015-16,82,242.1,36.8,82.1,0.448,7.9,22.7,0.347,28.9,59.4,0.487,16.8,22.6,0.743,9.3,31.8,41,21.4,7.2,4.4,13.3,20.6,98.3,CLE,2015-16,Chicago Bulls,1.0
1,2015-10-27,Detroit Pistons,106,Atlanta Hawks,94,82,241.8,38.6,84.4,0.458,9.9,28.4,0.35,28.7,56.1,0.512,15.6,20,0.783,8.3,33.8,42.1,25.6,9.1,5.9,15,19.1,102.8,ATL,2015-16,82,241.8,37.1,86.1,0.432,8.3,24.5,0.338,28.9,61.6,...,25.5,0.668,12.5,33.9,46.3,19.4,7,3.7,13.5,19,102,DET,2015-16,82,242.4,38.8,84.2,0.461,7.3,20.5,0.355,31.5,63.7,0.495,16.5,21,0.783,8.8,33.7,42.5,21.5,7.1,4.5,13.4,21.6,101.4,DET,2015-16,Detroit Pistons,0.0
2,2015-10-27,New Orleans Pelicans,95,Golden State Warriors,111,82,242.4,42.5,87.3,0.487,13.1,31.6,0.416,29.4,55.7,0.528,16.7,21.8,0.763,10,36.2,46.2,28.9,8.4,6.1,15.2,20.7,114.9,GSW,2015-16,82,242.4,38.9,89.4,0.435,7.8,23.5,0.332,31.1,65.9,...,22.2,0.776,9.5,33.1,42.6,22.2,7.7,4.2,13.4,20.9,102.7,NOP,2015-16,82,241.2,39.2,83.9,0.468,9.2,24.9,0.369,30,59,0.51,18.8,24,0.783,8.9,35.5,44.4,23.2,7.3,5.2,13.7,19.9,106.5,NOP,2015-16,Golden State Warriors,1.0
3,2015-10-28,Washington Wizards,88,Orlando Magic,87,82,242.7,39.5,86.8,0.455,7.8,22.2,0.35,31.8,64.7,0.492,15.2,20.1,0.757,10.3,33,43.3,23.6,8.2,5.1,14.1,20.7,102.1,ORL,2015-16,82,242.7,38.4,83.5,0.46,8.9,24.9,0.359,29.5,58.6,...,22.5,0.73,9.1,32.8,41.8,24.5,8.6,3.9,14.5,20.8,104.1,WAS,2015-16,82,240.9,38.7,83.8,0.462,9,24.2,0.371,29.7,59.5,0.498,18.3,23.7,0.769,9.4,34.9,44.3,22.5,8.1,4.3,16.2,20.1,104.6,WAS,2015-16,Washington Wizards,0.0
4,2015-10-28,Philadelphia 76ers,95,Boston Celtics,112,82,241.2,39.2,89.2,0.439,8.7,26.1,0.335,30.5,63.1,0.483,18.5,23.5,0.788,11.6,33.3,44.9,24.2,9.2,4.2,13.7,21.9,105.7,BOS,2015-16,82,241.2,37.5,85,0.441,7.8,23.3,0.336,29.7,61.7,...,22.6,0.694,9.5,31.8,41.2,21.5,8.3,6,16.4,21.7,97.4,PHI,2015-16,82,241.5,39.7,85.4,0.464,7.8,21.7,0.359,31.9,63.8,0.5,20.5,26.1,0.786,11.1,36.4,47.6,23.4,8.7,5.7,15.1,19.2,107.6,PHI,2015-16,Boston Celtics,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3439,2018-04-11,Washington Wizards,92,Orlando Magic,101,82,240.6,38.8,85.9,0.452,10.3,29.3,0.351,28.5,56.5,0.504,15.5,20.5,0.757,8.8,32.8,41.6,23.4,7.6,4.9,14.5,19.3,103.4,ORL,2017-18,82,240.6,40.9,87.5,0.468,10,27.6,0.362,30.9,60,...,21.8,0.772,10,33.1,43.1,25.2,7.9,4.3,14.6,21.3,106.6,WAS,2017-18,82,242.1,39,84.4,0.462,10.2,29.2,0.349,28.8,55.2,0.522,17.9,23.5,0.761,9.8,32.7,42.5,22.3,7.8,4.7,14.9,20,106,WAS,2017-18,Washington Wizards,0.0
3440,2018-04-11,Milwaukee Bucks,95,Philadelphia 76ers,130,82,241.2,40.8,86.6,0.472,11,29.8,0.369,29.9,56.7,0.526,17.1,22.8,0.752,10.9,36.5,47.4,27.1,8.3,5.1,16.5,22.1,109.8,PHI,2017-18,82,241.2,38,87.6,0.434,10.1,29.6,0.342,27.9,58,...,23.4,0.783,8.4,31.5,39.8,23.2,8.8,5.4,13.8,21.4,106.5,MIL,2017-18,82,241.8,39.1,83.7,0.468,10.1,27.1,0.372,29,56.6,0.513,18.4,23.5,0.783,10,32.8,42.7,23.6,7.1

In [ ]:
box_df.to_csv('train_set.csv')
box_df5.to_csv('test_set.csv')

In [ ]:
print(box_df.isnull().sum(axis=0).sort_values(ascending=False))

Class       0
PF_x        0
TEAM_x      0
SEASON_x    0
OPP_G_x     0
           ..
BLK_y       0
TOV_y       0
PF_y        0
PTS_y       0
DATE        0
Length: 107, dtype: int64


In [ ]:
train = pd.read_csv('test_set.csv')
train

,Unnamed: 0,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS,G_x,MP_x,FG_x,FGA_x,FG%_x,3P_x,3PA_x,3P%_x,2P_x,2PA_x,2P%_x,FT_x,FTA_x,FT%_x,ORB_x,DRB_x,TRB_x,AST_x,STL_x,BLK_x,TOV_x,PF_x,PTS_x,TEAM_x,SEASON_x,OPP_G_x,OPP_MP_x,OPP_FG_x,OPP_FGA_x,OPP_FG%_x,OPP_3P_x,OPP_3PA_x,OPP_3P%_x,OPP_2P_x,...,FTA_y,FT%_y,ORB_y,DRB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y,PTS_y,TEAM_y,SEASON_y,OPP_G_y,OPP_MP_y,OPP_FG_y,OPP_FGA_y,OPP_FG%_y,OPP_3P_y,OPP_3PA_y,OPP_3P%_y,OPP_2P_y,OPP_2PA_y,OPP_2P%_y,OPP_FT_y,OPP_FTA_y,OPP_FT%_y,OPP_ORB_y,OPP_DRB_y,OPP_TRB_y,OPP_AST_y,OPP_STL_y,OPP_BLK_y,OPP_TOV_y,OPP_PF_y,OPP_PTS_y,TEAM_y.1,SEASON_y.1,winning_name,Class
0,0,2018-10-16,Philadelphia 76ers,87,Boston Celtics,105,82,241.2,42.1,90.5,0.465,12.6,34.5,0.365,29.5,56.0,0.527,15.6,19.5,0.802,9.8,34.7,44.5,26.3,8.6,5.3,12.8,20.4,112.4,BOS,2018-19,82,241.2,39.5,88.1,0.448,11.5,33.5,0.344,28.0,...,27.5,0.771,10.9,36.9,47.8,26.9,7.4,5.3,14.9,21.3,115.2,PHI,2018-19,82,241.5,41.7,91.5,0.455,10.3,30.0,0.342,31.4,61.5,0.511,18.8,24.5,0.768,10.0,33.5,43.5,23.4,7.7,4.1,12.7,22.1,112.5,PHI,2018-19,Boston Celtics,1.0
1,1,2018-10-16,Oklahoma City Thunder,100,Golden State Warriors,108,82,241.5,44.0,89.8,0.491,13.3,34.4,0.385,30.8,55.3,0.557,16.3,20.4,0.801,9.7,36.5,46.2,29.4,7.6,6.4,14.3,21.4,117.7,GSW,2018-19,82,241.5,40.4,91.1,0.444,11.6,33.6,0.347,28.8,...,25.0,0.713,12.6,35.5,48.1,23.4,9.3,5.2,14.0,22.4,114.5,OKC,2018-19,82,242.1,40.8,88.7,0.460,11.2,31.6,0.356,29.5,57.1,0.517,18.3,24.0,0.761,9.9,35.8,45.7,24.9,8.2,5.1,16.7,22.5,111.1,OKC,2018-19,Golden State Warriors,1.0
2,2,2018-10-17,Brooklyn Nets,100,Detroit Pistons,103,82,242.1,38.8,88.3,0.440,12.1,34.8,0.348,26.7,53.5,0.500,17.3,23.1,0.747,11.4,33.6,45.0,22.5,6.9,4.0,13.8,22.1,107.0,DET,2018-19,82,242.1,39.9,85.0,0.469,9.6,27.8,0.345,30.3,...,25.5,0.745,11.0,35.6,46.6,23.8,6.6,4.1,15.1,21.5,112.2,BRK,2018-19,82,243.7,42.2,92.5,0.456,10.3,30.3,0.341,31.9,62.1,0.513,17.6,22.8,0.770,11.0,35.2,46.2,23.7,7.8,5.3,13.5,22.0,112.3,BRK,2018-19,Detroit Pistons,1.0
3,3,2018-10-17,Memphis Grizzlies,83,Indiana Pacers,111,82,240.3,41.3,87.0,0.475,9.5,25.4,0.374,31.8,61.6,0.517,15.8,21.1,0.752,9.3,33.7,43.0,26.0,8.7,4.9,13.7,19.4,108.0,IND,2018-19,82,240.3,38.7,86.1,0.450,11.4,32.1,0.354,27.4,...,23.0,0.772,8.8,33.0,41.8,23.9,8.3,5.5,14.0,22.0,103.5,MEM,2018-19,82,242.4,37.6,83.4,0.451,11.6,32.2,0.360,26.0,51.2,0.508,19.3,24.7,0.784,9.5,35.2,44.7,23.4,7.6,4.9,15.2,21.4,106.1,MEM,2018-19,Indiana Pacers,1.0
4,4,2018-10-17,Miami Heat,101,Orlando Magic,104,82,241.2,40.4,89.1,0.454,11.4,32.1,0.356,29.0,57.0,0.509,15.0,19.2,0.782,10.0,35.4,45.4,25.5,6.6,5.4,13.2,18.6,107.3,ORL,2018-19,82,241.2,39.9,87.6,0.456,10.5,30.2,0.347,29.5,...,21.7,0.695,11.2,35.1,46.3,24.3,7.6,5.5,14.7,20.9,105.7,MIA,2018-19,82,240.6,38.3,86.8,0.441,11.9,33.1,0.358,26.4,53.6,0.493,17.5,22.7,0.769,10.1,34.0,44.2,23.3,7.5,4.7,14.1,20.1,105.9,MIA,2018-19,Orlando Magic,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1143,1143,2019-04-10,Chicago Bulls,109,Philadelphia 76ers,125,82,241.5,41.5,88.2,0.471,10.8,30.2,0.359,30.7,58.0,0.529,21.2,27.5,0.771,10.9,36.9,47.8,26.9,7.4,5.3,14.9,21.3,115.2,PHI,2018-19,82,241.5,41.7,91.5,0.455,10.3,30.0,0.342,31.4,...,20.7,0.783,8.8,34.1,42.9,21.9,7.4,4.3,14.1,20.3,104.9,CHI,2018-19,82,242.7,42.2,89.2,0.473,12.1,33.0,0.365,30.2,56.2,0.537,16.8,22.0,0.763,10.0,36.4,46.4,25.5,7.3,5.8,13.3,18.7,113.4,CHI,2018-19,Philadelphia 76ers,1.0
1144,1144,2019-04-10,Dallas Mavericks,94,San Antonio Spurs,105,82,241.5,42.3,88.4,0.478,9.9,25.3,0.392,32.4,63.1,0.513,17.2,21.0,0.819,9.2,35.5,44.7,24.5,6.1,4.7,12.1,18.1,111.7,SAS,2018-19,82,241.5,41.5,89.7,0.463,11.7,32.7,0.359,29.8,...,25.3,0.742,10.1,35.2,45.3,23.4,6.5,4.3,14.2,20.1,108.9,DAL,2018-19,82,241.2,40.9,89.2,0.459,11.2,3

In [ ]:
train = train.fillna(0)
y = train.Class
x = train.drop(columns=['Class','DATE','VISITOR','HOME','TEAM_x','SEASON_x','TEAM_y.1','SEASON_y.1',
                        'winning_name','TEAM_y','SEASON_y','TEAM_x.1','SEASON_x.1','OPP_G_x', 'G_x', 'G_y', 'OPP_G_y',
                        'MP_x', 'OPP_MP_x', 'OPP_MP_y', 'MP_y', 'OPP_ORB_x', 'OPP_AST_x', 'OPP_TOV_x', 'OPP_PTS_x', 'DRB_y', 'STL_y', 'BLK_y', 'PF_y', 'OPP_ORB_y', 'OPP_AST_y', 'OPP_TOV_y', 'OPP_PTS_y'])
y = y.fillna(0)
x = x.fillna(0)

In [ ]:
x

In [ ]:
x = x.values
y = y.values

In [ ]:
def get_model():
    # define the keras model
    model = keras.Sequential()
    model.add(layers.Dense(100, input_dim=x.shape[1], activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(25, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

model = get_model()

In [ ]:
def train_model(x_train, y_train):
    model = get_model()
    model.compile(optimizer='adam', loss='BinaryCrossentropy', metrics=['accuracy'])
    er = EarlyStopping(patience=50, min_delta=1e-4, restore_best_weights=True, monitor='loss')
    model.fit(x_train, y_train, epochs=50, callbacks=[er], batch_size=32)
    return model

In [ ]:
kFold_CV = RepeatedKFold(n_splits=5, n_repeats=5)
models = []
x = np.asarray(x).astype('float32')
for index, val in kFold_CV.split(x, y):    
    xtrain, ytrain = x[index], y[index]
    model = train_model(xtrain, ytrain)
    models.append(model)

Epoch 1/50
29/29 [==============================] - 0s 2ms/step - loss: 7.2209 - accuracy: 0.5033
Epoch 2/50
29/29 [==============================] - 0s 1ms/step - loss: 2.9172 - accuracy: 0.5458
Epoch 3/50
29/29 [==============================] - 0s 2ms/step - loss: 1.9414 - accuracy: 0.4978
Epoch 4/50
29/29 [==============================] - 0s 2ms/step - loss: 1.2640 - accuracy: 0.5501
Epoch 5/50
29/29 [==============================] - 0s 2ms/step - loss: 1.0960 - accuracy: 0.5305
Epoch 6/50
29/29 [==============================] - 0s 2ms/step - loss: 0.9538 - accuracy: 0.5479
Epoch 7/50
29/29 [==============================] - 0s 2ms/step - loss: 0.9750 - accuracy: 0.5447
Epoch 8/50
29/29 [==============================] - 0s 2ms/step - loss: 0.8744 - accuracy: 0.5316
Epoch 9/50
29/29 [==============================] - 0s 2ms/step - loss: 0.8335 - accuracy: 0.5490
Epoch 10/50
29/29 [==============================] - 0s 2ms/step - loss: 0.8134 - accuracy: 0.5359
Epoch 11/50
29/29 [

In [ ]:
acc = []
for model in models:
    acc.append(np.mean(model.history.history['accuracy']))

print(np.mean(acc))

0.5825803996801376
